In [ ]:
import pickle,os
import os.path as osp
import re
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()
from ohbm.api import Api
import ohbm

In [ ]:
access_token="Ask OHBM"
url='https://ww5.aievolution.com/hbm1701'
api = Api(access_token,url)

In [ ]:
import sys
import hashlib
# BUF_SIZE is totally arbitrary, change for your app!
BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

def get_hash(filename, hashfunc=hashlib.sha1()):
    with open(filename, 'rb') as f:
        data = None
        while True:
            data = f.read(BUF_SIZE)
            if not data:
                break
            hashfunc.update(data)

    return hashfunc.hexdigest()

#def _get_hash():
#    if hashtype == 'sha1': 
#        sha1 = hashlib.sha1()
#    else 
#        md5 = hashlib.md5()

In [ ]:
def hash_of_file(filepath,hashfunc='sha1'):
    import hashlib
    BUFSIZE = 2**10
    if hashfunc == 'sha1':
        hashf = hashlib.sha1()
    with open(filepath, 'rb') as f:
        while True:
            block = f.read(BUFSIZE) # Magic number: one-megabyte blocks.
            if not block: break
            hashf.update(block)
        return hashf.hexdigest()

In [ ]:
pckg_dir = osp.dirname(osp.dirname(ohbm.__file__))
data_dir = osp.join(pckg_dir, 'abstracts_data')
abstract_pkl = osp.join(data_dir, 'abstracts.pkl')

In [ ]:
ABST_SHA1 = 'e8047573ad5f6b1f6aa079bf4de874f0cf53c969'

try:
    abstracts=pickle.load(open('abstracts.pkl','rb'))
    # assert hashlib.sha1(open(abstract_pkl).read()) == ABST_SHA1
    assert (ABST_SHA1 == hash_of_file(abstract_pkl))
except:    
    abstracts = api.Abstracts.getAbstracts()
    pickle.dump(abstracts, open(osp.join(data_dir, 'abstracts.pkl'),'wb'))


In [ ]:
a = abstracts[0]

In [ ]:
a.keys()

### Make a list of document, each is an abstract

In [ ]:
fields=['title','results'] #,'materialsMethods' 'conclusion','title', ,'results'] # ,'purpose',]
doc_list = []
bad_docu = 0
digit_regexp=re.compile('\d')
bad_code_regexp=re.compile('\W')

for a in abstracts: # abstracts[:300]:
    #d = [a[f] for f in fields]
    # remove document that contain None: they dont have strings for all fields
    #if None in d:
    #    continue   
    try:
        alltext = ' '.join([a[f] for f in fields])
    except TypeError:
        bad_docu += 1
        continue
    
    alltext=re.sub(digit_regexp,' ',alltext)
    alltext=re.sub(bad_code_regexp,' ',alltext)
    
    # d = [s for s in d if s] # remove None from list - may be remove abstract ?
    # print(d); print("\n")
    #doc_list.append(' '.join(alltext))
    doc_list.append(alltext)

In [ ]:
doc_list[1]
print(len(doc_list))
print(bad_docu)

In [ ]:
#doc_list[2]

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim import corpora, models


tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

In [ ]:
len(en_stop)
en_stop[:3]
other_stops = ['p','left','right','brain','hemisphere',
               'figure','significant','fig','activation', 'cluster', 'results',
              'showed', 'based', 'may', 'found', 'subjects', 'age', 'aged', 'hiv', 
               'data', 'compared', 'matter', 'larger', 'smaller', 'sub', 'values',
              'analysis','c','negative', 'positive','region','group','cortex','activity',
              'areas','processing','using','cortical','et','al','sample','state','white',
               'participants','available','mm','ms','image','task','can',
              'across','studies','gm','wm','neuroimaging','gyrus','volume','method',
               'condition','conditions','methods','method','model','fmri','mri','head',
              'regions','will','regions','tr','approach','subject','peculiar','also',
               'research','mean','non','signal','different','users','user','te','related',
              'performed','perform', 'images', 'echo', 'one', 'two', 'three', 'four','five',
              'six', 'seven', 'eight', 'nine', 'ten', 'eleven','twelve','use','used','new',
              'dataset','datasets','associated','size','output','outputs','map','maps',
                'study', 'studies', 'change', 'changes','en','effect','effects','differences',
              'difference','voxels','voxel','time','times','score','scores','increased',
              'increase','weight','weights','voxel',
              'voxels','acquired','acquisition','acquisitions','whether','presented',
              'measure', 'measured','enable','considering','described','describe','included',
              'include','shows','showed','show']
en_stop += other_stops
en_stop += [letter for letter in 'qwertyuiopasdfghjklzxcvbnm1234567890']

In [ ]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_list:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    #texts.append(stopped_tokens)
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

In [ ]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=25, 
                                           id2word = dictionary, passes=20)

In [ ]:
print(ldamodel.print_topics(num_topics=25, num_words=7))

In [ ]:
print(len(ldamodel.id2word.keys()))
ldamodel.id2word[ldamodel.id2word.keys()[500]]
print(texts[0])
ldamodel.minimum_probability

In [ ]:
print(len(corpus))
print(corpus[100])
ldamodel.minimum_probability = .01
print(ldamodel[corpus[100]])
ldamodel.minimum_probability = .001
print(ldamodel[corpus[100]])

ldamodel.num_topics/len(corpus)
1./ldamodel.num_topics
#ldamodel[texts[0]]

In [ ]:
for tid in range(3): #ldamodel.num_topics):
    topic = ldamodel.show_topic(tid,topn=50)
    print(topic)